<a href="https://colab.research.google.com/github/EMSIMa/ADD3IIR/blob/main/08_It%C3%A9rateurs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Itérateurs

Souvent, un élément important de l'analyse des données consiste à répéter un calcul similaire, encore et encore, de manière automatisée.
Par exemple, vous pouvez avoir un tableau de noms que vous aimeriez diviser en prénom et nom de famille, ou peut-être des dates que vous aimeriez convertir dans un format standard.
L'une des réponses de Python à ce problème est la syntaxe *iterator*.
Nous l'avons déjà vu avec l'itérateur ``range`` :

In [ ]:
for i in range(10):
    print(i, end=' ')

0 1 2 3 4 5 6 7 8 9 

Ici, nous allons creuser un peu plus loin.
Il s'avère que dans Python 3, ``range`` n'est pas une liste, mais quelque chose appelé *iterator*, et apprendre comment il fonctionne est la clé pour comprendre une large classe de fonctionnalités très utiles de Python.

## Itérer sur des listes
Les itérateurs sont peut-être plus faciles à comprendre dans le cas concret de l'itération sur une liste.
Considérons ce qui suit :

In [ ]:
for value in [2, 4, 6, 8, 10]:
    # do some operation
    print(value + 1, end=' ')

3 5 7 9 11 

La syntaxe familière " ``for x in y`` " nous permet de répéter une opération pour chaque valeur de la liste.
Le fait que la syntaxe du code soit si proche de sa description en anglais ("*for [each] value in [the] list*") n'est qu'un des choix syntaxiques qui font de Python un langage si intuitif à apprendre et à utiliser.

Mais ce n'est pas le comportement qui se produit *réellement*.
Quand vous écrivez quelque chose comme ``for val in L``, l'interpréteur Python vérifie s'il a une interface *iterator*, ce que vous pouvez vérifier vous-même avec la fonction intégrée ``iter`` :

In [ ]:
iter([2, 4, 6, 8, 10])

C'est cet objet iterator qui fournit la fonctionnalité requise par la boucle ``for``.
L'objet ``iter`` est un conteneur qui vous donne accès à l'objet suivant tant qu'il est valide, ce qui peut être vu avec la fonction intégrée ``next`` :

In [ ]:
I = iter([2, 4, 6, 8, 10])

In [ ]:
print(next(I))

2


In [ ]:
print(next(I))

4


In [ ]:
print(next(I))

6


Quel est le but de ce niveau d'indirection ?
Eh bien, il s'avère que c'est incroyablement utile, parce que cela permet à Python de traiter les choses comme des listes alors qu'elles ne sont *pas réellement des listes*.

## ``range()`` : Une liste n'est pas toujours une liste
L'exemple le plus courant de cette itération indirecte est peut-être la fonction ``range()``, qui renvoie non pas une liste, mais un objet spécial ``range()`` :

In [ ]:
range(10)

range(0, 10)

``range``, comme une liste, expose un itérateur :

In [ ]:
iter(range(10))

Python sait donc qu'il faut le traiter *comme si* il s'agissait d'une liste :

In [ ]:
for i in range(10):
    print(i, end=' ')

0 1 2 3 4 5 6 7 8 9 

L'avantage de l'indirection de l'itérateur est que *la liste complète n'est jamais explicitement créée !*
Pour s'en convaincre, il suffit d'effectuer un calcul de plage qui saturerait la mémoire de notre système si nous l'instancions réellement 

In [ ]:
N = 10 ** 12
for i in range(N):
    if i >= 10: break
    print(i, end=', ')

0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 

Si ``range`` devait réellement créer cette liste de mille milliards de valeurs, elle occuperait des dizaines de téraoctets de mémoire machine : un gaspillage, étant donné que nous ignorons toutes les valeurs sauf les 10 premières !

En fait, il n'y a aucune raison pour que les itérateurs se terminent !
La bibliothèque ``itertools`` de Python contient une fonction ``count`` qui agit comme une plage infinie :

In [ ]:
from itertools import count

for i in count():
    if i >= 10:
        break
    print(i, end=', ')

0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 

Si nous n'avions pas introduit une interruption de boucle ici, le processus continuerait à compter joyeusement jusqu'à ce qu'il soit interrompu manuellement ou tué (en utilisant, par exemple, ``ctrl-C``).

## Itérateurs utiles
Cette syntaxe d'itérateur est utilisée presque universellement dans les types intégrés de Python ainsi que dans les objets plus spécifiques à la science des données que nous explorerons dans les sections suivantes.
Nous aborderons ici quelques-uns des itérateurs les plus utiles du langage Python :

### ``enumerate``
Il est souvent nécessaire d'itérer non seulement les valeurs d'un tableau, mais aussi de garder une trace de l'index.
Vous pourriez être tenté de faire les choses de cette façon :

In [ ]:
L = [2, 4, 6, 8, 10]
for i in range(len(L)):
    print(i, L[i])

0 2
1 4
2 6
3 8
4 10


Bien que cela fonctionne, Python fournit une syntaxe plus propre en utilisant l'itérateur ``enumerate`` :

In [ ]:
for i, val in enumerate(L):
    print(i, val)

0 2
1 4
2 6
3 8
4 10


C'est la façon la plus "pythonique" d'énumérer les indices et les valeurs d'une liste.

### ``Zip``
Dans d'autres cas, vous pouvez avoir plusieurs listes sur lesquelles vous voulez itérer simultanément.
Vous pourriez certainement itérer sur l'index comme dans l'exemple non pythonique que nous avons vu précédemment, mais il est préférable d'utiliser l'itérateur ``zip``, qui regroupe les itérables :

In [ ]:
L = [2, 4, 6, 8, 10]
R = [3, 6, 9, 12, 15]
for lval, rval in zip(L, R):
    print(lval, rval)

2 3
4 6
6 9
8 12
10 15


N'importe quel nombre d'itérables peut être zippé ensemble, et s'ils sont de longueurs différentes, le plus court déterminera la longueur du ``zip``.

### ``map`` et ``filter``
L'itérateur ``map`` prend une fonction et l'applique aux valeurs de l'itérateur :

In [ ]:
# find the first 10 square numbers
square = lambda x: x ** 2
for val in map(square, range(10)):
    print(val, end=' ')

0 1 4 9 16 25 36 49 64 81 

L'itérateur ``filter`` est similaire, sauf qu'il ne passe que les valeurs pour lesquelles la fonction de filtrage est évaluée à True :

In [ ]:
# find values up to 10 for which x % 2 is zero
is_even = lambda x: x % 2 == 0
for val in filter(is_even, range(10)):
    print(val, end=' ')

0 2 4 6 8 

Les fonctions ``map`` et ``filter``, ainsi que la fonction ``reduce`` (qui se trouve dans le module ``functools`` de Python) sont des composants fondamentaux du style de *programmation fonctionnelle*, qui, bien qu'il ne s'agisse pas d'un style de programmation dominant dans le monde Python, a ses fervents partisans (voir, par exemple, la bibliothèque [pytoolz](https://toolz.readthedocs.org/en/latest/)).

### Les itérateurs comme arguments de fonction

Nous avons vu que ``*args`` et ``**kwargs`` peuvent être utilisés pour passer des séquences et des dictionnaires aux fonctions.
Il s'avère que la syntaxe ``*args`` ne fonctionne pas seulement avec les séquences, mais avec n'importe quel itérateur :

In [ ]:
print(*range(10))

0 1 2 3 4 5 6 7 8 9


Ainsi, par exemple, nous pouvons comprimer l'exemple ``map`` de tout à l'heure en ce qui suit :

In [ ]:
print(*map(lambda x: x ** 2, range(10)))

0 1 4 9 16 25 36 49 64 81


L'utilisation de cette astuce nous permet de répondre à l'éternelle question qui revient dans les forums d'apprenants de Python : pourquoi n'y a-t-il pas de fonction ``unzip()`` qui fait le contraire de ``zip()`` ?
Si vous vous enfermez dans un placard et que vous y réfléchissez un peu, vous pourriez réaliser que le contraire de ``zip()`` est... ``zip()`` ! La clé est que ``zip()`` peut rassembler n'importe quel nombre d'itérateurs ou de séquences. Observez :

In [ ]:
L1 = (1, 2, 3, 4)
L2 = ('a', 'b', 'c', 'd')

In [ ]:
z = zip(L1, L2)
print(*z)

(1, 'a') (2, 'b') (3, 'c') (4, 'd')


In [ ]:
z = zip(L1, L2)
new_L1, new_L2 = zip(*z)
print(new_L1, new_L2)

(1, 2, 3, 4) ('a', 'b', 'c', 'd')


Réfléchissez un peu. Si vous comprenez pourquoi cela fonctionne, vous aurez fait un grand pas dans la compréhension des itérateurs Python !

## Les itérateurs spécialisés : ``itertools``

Nous avons brièvement regardé l'itérateur infini ``range``, ``itertools.count``.
Le module ``itertools`` contient un grand nombre d'itérateurs utiles ; cela vaut la peine d'explorer le module pour voir ce qui est disponible.
A titre d'exemple, considérons la fonction ``itertools.permutations``, qui itère sur toutes les permutations d'une séquence :

In [ ]:
from itertools import permutations
p = permutations(range(3))
print(*p)

(0, 1, 2) (0, 2, 1) (1, 0, 2) (1, 2, 0) (2, 0, 1) (2, 1, 0)


De même, la fonction ``itertools.combinations`` itère sur toutes les combinaisons uniques de ``N`` valeurs dans une liste :

In [ ]:
from itertools import combinations
c = combinations(range(4), 2)
print(*c)

(0, 1) (0, 2) (0, 3) (1, 2) (1, 3) (2, 3)


L'itérateur ``produit``, qui itère sur tous les ensembles de paires entre deux ou plusieurs itérables, joue un rôle similaire :

In [ ]:
from itertools import product
p = product('ab', range(3))
print(*p)

('a', 0) ('a', 1) ('a', 2) ('b', 0) ('b', 1) ('b', 2)


De nombreux autres itérateurs utiles existent dans ``itertools`` : la liste complète peut être trouvée, avec quelques exemples, dans la [documentation en ligne](https://docs.python.org/3.5/library/itertools.html).